<a href="https://colab.research.google.com/github/soulsharp/SuperpixelGen-using-SLIC-MT/blob/main/SLIC_Algorithm_Fast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 35.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.8/92.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 9.2 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2025.1-cp311-cp311-linux_x86_64.whl size=660394 sha256=fcb7191c5609200f9b4a33a02d2be9386596ea1b57e69ab3bac65d1f9fcfa43f
  Stored in directory: /root/.cache/pip/wheels/77/7e/6c/d2d1451ea6424cdc3d67b36c16fa7111eafdf2034bc3405666
Successfully built pycuda


In [ ]:
import math
from skimage import io, color
from skimage.transform import resize
import numpy as np
import time
import pstats
import cProfile
import pycuda.driver as cuda
import pycuda.autoinit
import numpy as np
from pycuda.compiler import SourceModule

In [ ]:
# function which returns an object of class SuperPixel
def make_superPixel(h, w, img, superpixel_id):
    return SuperPixels(h, w, img[h, w][0], img[h, w][1], img[h, w][2], superpixel_id)

In [ ]:
# To define the initial cluster centers distanced at S
def initial_cluster_center(S, img, img_h, img_w):
    clusters = []
    h = S // 2
    w = S // 2
    while h < img_h:
        while w < img_w:
            num_superpixels = 0
            clusters.append(make_superPixel(h, w, img, num_superpixels))
            w += S
        w = S // 2
        h += S
    return clusters

In [ ]:
# function to calculate gradient at each pixel
def calc_gradient(h, w, img, img_w, img_h):
    if w + 1 >= img_w:
        w = img_w - 2
    if h + 1 >= img_h:
        h = img_h - 2
    grad = img[w + 1, h + 1][0] - img[w, h][0] + img[w + 1, h + 1][1] - img[w, h][1] + img[w + 1, h + 1][2] - img[w, h][2]
    return grad

In [ ]:
# function which reassign the cluster center to the pixel having the lowest gradient
def reassign_cluster_center_acc_to_grad(clusters, img):
    for c in clusters:
        cluster_gradient = calc_gradient(c.h, c.w, img, img_w, img_h)
        for dh in range(-1, 2):
            for dw in range(-1, 2):
                H = c.h + dh
                W = c.w + dw
                new_gradient = calc_gradient(H, W, img, img_w, img_h)
                if new_gradient < cluster_gradient:
                    c.update(H, W, img[H,W][0], img[H,W][1], img[H,W][2])
                    c_gradient = new_gradient


In [ ]:
# To-do : Major bottleneck, rewrite in CUDA
# Ensure that acesses are vectorized/ parallel
'''
function to assign pixels to the nearest cluster using the Euclidean distance involving both color and spatial
proximity.
'''
def assign_pixels_to_cluster(clusters, S, img, img_h, img_w, tag, dis):
    for c in clusters:
        for h in range(c.h - 2 * S, c.h + 2 * S):
            if h < 0 or h >= img_h: continue
            for w in range(c.w - 2 * S, c.w + 2 * S):
                if w < 0 or w >= img_w: continue
                l, a, b = img[h,w]
                Dc = math.sqrt(math.pow(l - c.l, 2) + math.pow(a - c.a, 2) + math.pow(b - c.b, 2))
                Ds = math.sqrt(math.pow(h - c.h, 2) + math.pow(w - c.w, 2))
                D = math.sqrt(math.pow(Dc / m, 2) + math.pow(Ds / S, 2))
                if D < dis[h,w]:
                    if (h, w) not in tag:
                        tag[(h, w)] = c
                        c.pixels.append((h, w))
                    else:
                        tag[(h, w)].pixels.remove((h, w))
                        tag[(h, w)] = c
                        c.pixels.append((h, w))
                    dis[h, w] = D

In [ ]:
# function to replace the cluster center with the mean of the pixels contained in the cluster
def update_cluster_mean(clusters):
    for c in clusters:
        sum_h = sum_w = number = 0
        #print("c.pixels",c.pixels)
        for p in c.pixels:
            sum_h += p[0]
            sum_w += p[1]
            number += 1
            H = sum_h // number
            W = sum_w // number
            c.update(H, W,img[H, W][0], img[H, W][1], img[H, W][2])

In [ ]:
# replace the color of each pixel in a cluster by the color of the cluster's center
def avg_color_cluster(img, name, clusters):
    image = np.copy(img)
    for c in clusters:
        for p in c.pixels:
            image[p[0],p[1]][0] = c.l
            image[p[0],p[1]][1] = c.a
            image[p[0],p[1]][2] = c.b
        # To change the color of cluster center to Black
        # image[c.h, c.w][0] = 0
        # image[c.h, c.w][1] = 0
        # image[c.h, c.w][2] = 0
    lab2rgb(name, image)

In [ ]:
# function for the Simple Linear Iterative Clustering
def slic(S, img, img_h, img_w, clusters, tag, dis):
    clusters = initial_cluster_center(S, img, img_h, img_w)
    reassign_cluster_center_acc_to_grad(clusters, img)

    # Runs SLIC for 10 iterations
    for i in range(10):
        assign_pixels_to_cluster(clusters, S, img, img_h, img_w, tag, dis)
        update_cluster_mean(clusters)

        # Saves image after 10 iterations
        if i == 9 :
            name = 'out_m{m}_k{k}.png'.format(loop=i, m=m, k=k)
            avg_color_cluster(img, name, clusters)
    return clusters

In [ ]:
# function to convert LAB images back to RGB and save it
def lab2rgb(path, lab_arr):
    rgb_arr = color.lab2rgb(lab_arr)
    rgb_arr = (rgb_arr * 255).astype(np.uint8)
    io.imsave(path, rgb_arr)

In [ ]:
# Todo - Assign a cluster id to each superpixel, which makes it easy to assign centers later
# A class to initialize the super pixels, of the form - [h,w,l,a,b].
class SuperPixels(object):

    def __init__(self, h, w, l=0, a=0, b=0, num_pixels=0):
        self.update(h, w, l, a, b, id)
        self.pixels = []

    def update(self, h, w, l, a, b, num_pixels):
        self.h = h
        self.w = w
        self.l = l
        self.a = a
        self.b = b
        self.num_pixels = num_pixels

In [ ]:
# Contains all kernels to make SLIC faster
cuda_kernel="""
#include <cfloat>

// Define a struct for all cluster centers
struct __align__(8) clusterCenter{
  float l;
  float a;
  float b;
  float x;
  float y;
  float numPixels;
};

// This kernel calculates the distance between a pixel and cluster centers and
// assigns them an ID based on the nearest cluster
__global__ void assignClusterCenters(float* L, float* A, float* B, clusterCenter* clusterPtr,
                                    int* labelMap, int imageWidth, int imageHeight,
                                    int numSuperpixels, int M, int S){

  extern __shared__ float clusterInfo[];

  int tid = threadIdx.x;
  int global_tid = tid + blockIdx.x * blockDim.x;

  // Memory bounds check
  if (global_tid >= imageWidth * imageHeight) return;

  // Total number of values to load
  int numValues = numSuperpixels * 6;

  // Workload per thread
  int elementsPerThread = (numValues + blockDim.x - 1) / blockDim.x;
  int startIdx = tid * elementsPerThread;

  // Loads X elements per thread into shared memory
  for (int i = 0; i < elementsPerThread; ++i) {
    int idx = startIdx + i;
    if (idx < numValues) {
      clusterInfo[idx] = reinterpret_cast<float*>(clusterPtr)[idx];
    }
  }

  __syncthreads();

  // Initializes utility pointers
  float* firstHalfPtr = clusterInfo;
  float* secondHalfPtr = &clusterInfo[numSuperpixels];
  float* clusterInfoPtr;
  float* updateBufferPtr;

  // Slic Algorithm operates for 10 iterations for convergence
  // For even cycles i.e. 0, 2, 4,...  the firstHalf serves as the clusterInfo, and the second half acts as the buffer to compute cluster centers for the next pass
  for(int k = 0; k < 10; ++k){
    int X = global_tid % imageWidth;
    int Y = global_tid / imageWidth;
    int minClusterIdx = 0;
    float minDist = FLT_MAX;

    if(k % 2 == 0){
      clusterInfoPtr = firstHalfPtr;
      updateBufferPtr = secondHalfPtr;

      }
    else{
      clusterInfoPtr = secondHalfPtr;
      updateBufferPtr = firstHalfPtr;
    }

    // Sets update buffer ptr to 0 because its used to compute the new cluster centers
    for (int i = 0; i < elementsPerThread; ++i){
    int idx = startIdx + i;
    if (idx < numValues) {
      updateBufferPtr[idx] = 0.0f;
    }
  }

    __syncthreads();


    // computes distance to all cluster centers and assign pixel to the nearest one
    for(int i = 0; i < numSuperpixels; ++i){
      float cL = clusterInfoPtr[i * 6 + 0];
      float cA = clusterInfoPtr[i * 6 + 1];
      float cB = clusterInfoPtr[i * 6 + 2];
      float cX = clusterInfoPtr[i * 6 + 3];
      float cY = clusterInfoPtr[i * 6 + 4];

      // Euclidean distance
      float eucDist = sqrtf((cX - X) * (cX - X) + (cY - Y) * (cY - Y));

      // LAB color distance
      float labDist = sqrtf((cL - L[global_tid]) * (cL - L[global_tid]) +
                      (cA - A[global_tid]) * (cA - A[global_tid]) +
                      (cB - B[global_tid]) * (cB - B[global_tid]));

      // Total distance
      float totalDist = sqrtf((labDist / M) * (labDist / M) + (eucDist / S) * (eucDist / S));

      // Gets the nearest cluster's ID
      if(totalDist < minDist){
        minClusterIdx = i;
        minDist = totalDist;
    }
  }

    // Assigns labelMap location corresponding to the tid, the min cluster ID
    labelMap[global_tid] = minClusterIdx;
    int minClusterIdxStart = minClusterIdx * 6;

    // Add values to the updateBuffer for averaging later
    atomicAdd(&updateBufferPtr[minClusterIdxStart + 0], L[global_tid]);
    atomicAdd(&updateBufferPtr[minClusterIdxStart + 1], A[global_tid]);
    atomicAdd(&updateBufferPtr[minClusterIdxStart + 2], B[global_tid]);
    atomicAdd(&updateBufferPtr[minClusterIdxStart + 3], X);
    atomicAdd(&updateBufferPtr[minClusterIdxStart + 4], Y);
    atomicAdd(&updateBufferPtr[minClusterIdxStart + 5], 1.0f);

    __syncthreads();

    // Update the cluster center vals when all threads are done updating their respective cluster center vals
    if(global_tid < numSuperpixels){
      if (updateBufferPtr[global_tid * 6 + 5] > 0) {
        updateBufferPtr[global_tid * 6 + 0] /= updateBufferPtr[global_tid * 6 + 5];
        updateBufferPtr[global_tid * 6 + 1] /= updateBufferPtr[global_tid * 6 + 5];
        updateBufferPtr[global_tid * 6 + 2] /= updateBufferPtr[global_tid * 6 + 5];
        updateBufferPtr[global_tid * 6 + 3] /= updateBufferPtr[global_tid * 6 + 5];
        updateBufferPtr[global_tid * 6 + 4] /= updateBufferPtr[global_tid * 6 + 5];
      }
      updateBufferPtr[global_tid * 6 + 5] = 0.0;
    }

}


}

"""



In [ ]:
# Reads the input RGB image
rgb = io.imread("/content/joel-filipe-QwoNAhbmLLo-unsplash.jpg")
print(rgb.shape)

# Resize images to (400 x 400) for processing
img = resize(rgb, (400,400), anti_aliasing=True)
print(img.shape)

# convert RGB to LAB
img = color.rgb2lab(img)
L, A, B = img[:, :, 0].ravel(), img[:, :, 1].ravel(), img[:, :, 2].ravel()

k = 150   # Number of Super pixels
M = 20    # Constant for normalizing the color proximity, range of m = [1,40]

img_h = img.shape[0] # Image Height
img_w = img.shape[1] # Image Width

N = img_h * img_w  # Total number of pixels in the image
S = int(math.sqrt(N /k)) # average size of each superpixel

# clusters = []
# tag = {}
# # initialize the distance between pixels and cluster center as infinity
# dis = np.full((img_h, img_w), np.inf)

(4610, 3648, 3)
(400, 400, 3)


In [ ]:
L.size, L.itemsize

(160000, 8)

In [ ]:
# Gets the initial cluster center
clusters = initial_cluster_center(S, img, img_h, img_w)

# Defines a np dtype that contains all important info abt the cluster centers
struct_dtype = np.dtype([
  ("l", np.float32),
  ("a", np.float32),
  ("b", np.float32),
  ("x", np.float32),
  ("y", np.float32),
  ("numPixels", np.float32),
], align=True)

cluster_array = np.array([(cluster.l, cluster.a, cluster.b, cluster.h, cluster.w, 0.0) for cluster in clusters], dtype=struct_dtype)

In [ ]:
print(cluster_array)

[(19.13922  ,  7.8905888, -13.831687  ,  16.,  16., 0.)
 ( 5.660978 ,  7.3664594,  -8.591886  ,  16.,  48., 0.)
 (37.6379   ,  6.8000817, -15.309642  ,  16.,  80., 0.)
 (54.79511  ,  2.96065  ,  -7.700683  ,  16., 112., 0.)
 (12.899702 , 11.21094  ,  -7.313361  ,  16., 144., 0.)
 ( 7.657459 , 11.305485 ,  -7.8867354 ,  16., 176., 0.)
 ( 3.9409726,  6.802667 ,  -2.5436146 ,  16., 208., 0.)
 ( 4.7125497,  8.720329 ,  -8.256036  ,  16., 240., 0.)
 (16.818556 , 15.734594 ,  -8.051471  ,  16., 272., 0.)
 (10.187591 , 10.393866 ,  -8.2598295 ,  16., 304., 0.)
 (32.07077  , 19.167753 ,   3.1412327 ,  16., 336., 0.)
 ( 3.922091 ,  5.9159417,  -5.0802226 ,  16., 368., 0.)
 ( 5.016655 ,  4.851368 ,  -9.937559  ,  48.,  16., 0.)
 ( 3.5466824,  3.0492494,  -6.3172865 ,  48.,  48., 0.)
 ( 2.245238 ,  2.1932013,  -4.59636   ,  48.,  80., 0.)
 (13.932082 ,  8.576625 , -11.408364  ,  48., 112., 0.)
 (33.64469  ,  8.128026 ,  -8.934831  ,  48., 144., 0.)
 (12.561979 , 10.192486 , -12.1080675 ,  48., 17

In [ ]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
cuda.Device(0).get_attribute(cuda.device_attribute.MAX_SHARED_MEMORY_PER_BLOCK)//1024

48

In [ ]:
(150 * 6 * 2 * 4) // 1024

7

In [ ]:
try:
    mod = SourceModule(cuda_kernel, options=["-arch=sm_75", "--ptxas-options=-v"])
except Exception as e:
    print(e)

assign_clusters_fn = mod.get_function("assignClusterCenters")

# cluster_array is a np array that contains LABXY, ID for each superpixel center
cluster_info_size = cluster_array.nbytes
label_map = np.zeros(N, dtype=np.int32)
print(label_map.size, label_map.itemsize)


# size of L, A, B arrays in bytes
size_LAB = L.size * L.itemsize
size_label_map = label_map.size * label_map.itemsize

# print(size_LAB, size_label_map, label_map)

d_L = cuda.mem_alloc(size_LAB)
d_A = cuda.mem_alloc(size_LAB)
d_B = cuda.mem_alloc(size_LAB)
d_labelMap = cuda.mem_alloc(size_label_map)
d_clusters = cuda.mem_alloc(cluster_info_size)

#  Checks memory allocation
if d_L is None:
    raise RuntimeError("CUDA memory allocation failed for d_L!")

if d_A is None:
    raise RuntimeError("CUDA memory allocation failed for d_A!")

if d_B is None:
    raise RuntimeError("CUDA memory allocation failed for d_B!")

if d_labelMap is None:
    raise RuntimeError("CUDA memory allocation failed for d_labelMap!")

if d_clusters is None:
    raise RuntimeError("CUDA memory allocation failed for d_clusters!")


# Copies Data to Device
cuda.memcpy_htod(d_L, L)
cuda.memcpy_htod(d_A, A)
cuda.memcpy_htod(d_B, B)
cuda.memcpy_htod(d_labelMap, label_map)
cuda.memcpy_htod(d_clusters, cluster_array)

# CUDA Grid Configuration
threadsPerBlock = 256
numBlocks = (N + threadsPerBlock - 1) // threadsPerBlock

# Compute Shared Memory Size
sizeSmem = 2 * cluster_info_size

# Kernel Launch
assign_clusters_fn(d_L, d_A, d_B, d_clusters, d_labelMap,np.int32(img_w),
                   np.int32(img_h), np.int32(N), np.int32(M), np.int32(S),
                   block=(threadsPerBlock, 1, 1), grid=(numBlocks, 1),
                   shared=sizeSmem)

cuda.Context.synchronize()
print("Kernel executed successfully")

cuda.memcpy_dtoh(label_map, d_labelMap)


cuModuleLoadDataEx failed: an illegal memory access was encountered - 


LogicError: cuModuleGetFunction failed: an illegal memory access was encountered

In [ ]:
print(label_map.shape)

(160000,)


In [ ]:
np.unique(label_map, return_counts=True)

(array([0], dtype=int32), array([160000]))

In [ ]:
start_time = time.time()
cluster = slic(S, img, img_h, img_w, clusters, tag, dis)
end_time = time.time()

print(f"Time taken is {(end_time - start_time) * 1000} ms")

Time taken is 131309.42249298096 ms


In [ ]:
cProfile.run('slic(S, img, img_h, img_w, clusters, tag, dis)', 'profile_results')

In [ ]:
stats = pstats.Stats('profile_results')
stats.sort_stats(pstats.SortKey.TIME).print_stats(15)

Wed Feb  5 07:56:45 2025    profile_results

         830874310 function calls (830874307 primitive calls) in 578.676 seconds

   Ordered by: internal time
   List reduced from 299 to 15 due to restriction <15>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       10  446.984   44.698  574.103   57.410 <ipython-input-6-8cd683e35477>:5(assign_pixels_to_cluster)
580425832  100.727    0.000  100.727    0.000 {built-in method math.pow}
248753928   25.630    0.000   25.630    0.000 {built-in method math.sqrt}
       10    3.339    0.334    4.023    0.402 <ipython-input-7-2dcaf0d4e4fe>:2(update_cluster_mean)
    43385    0.757    0.000    0.757    0.000 {method 'remove' of 'list' objects}
  1601879    0.685    0.000    0.685    0.000 <ipython-input-11-cc6432300d62>:8(update)
        1    0.430    0.430    0.527    0.527 <ipython-input-8-d4992ef63ce6>:2(avg_color_cluster)
        1    0.029    0.029    0.038    0.038 /usr/local/lib/python3.11/dist-packages/skimage/co

In [ ]:
# superpixels
for c in cluster:
    print("H {} : W {}, l {}, a {}, b {}".format(c.h,c.w,c.l,c.a,c.b))

H 19 : W 20, l 24.089975801150572, a 6.983582114595571, b -13.266387089229582
H 16 : W 65, l 16.40778156165713, a 8.0141126927899, b -13.93056274157145
H 16 : W 112, l 54.79510797636068, a 2.960649997742648, b -7.700683087250404
H 25 : W 150, l 10.238819499505958, a 9.567046758129457, b -7.46135584808576
H 22 : W 185, l 3.403919588186259, a 4.9411272689862535, b -2.265624862039256
H 19 : W 219, l 12.324725907118658, a 9.593432142641815, b -0.5983476330680915
H 18 : W 257, l 8.092761208054739, a 10.681077429955735, b -9.075347388015853
H 19 : W 299, l 4.921299588520306, a 8.875064284386628, b -8.29229822245907
H 14 : W 339, l 6.623024157664659, a 10.432628886679742, b -8.98276346783824
H 19 : W 378, l 3.6456278365098953, a 5.468407041481507, b -4.6360393064611145
H 60 : W 19, l 5.104836088688547, a 4.198537100423519, b -8.945196661033378
H 54 : W 66, l 4.979812284699815, a 4.975935550010885, b -7.376025677730841
H 56 : W 106, l 21.377268316316183, a 6.909486083425426, b -13.645276743480